In [1]:
import pandas as pd
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

# Path ke folder dan file
BASE_DIR = "C:/Users/ASUS/Downloads/gammafest25"  # Ganti kalau file kamu di folder lain
PAPER_DIR = os.path.join(BASE_DIR, "Paper Database/Paper Database")

# Load CSV
train_df = pd.read_csv(os.path.join(BASE_DIR, "train.csv"))
metadata_df = pd.read_csv(os.path.join(BASE_DIR, "papers_metadata.csv"))

IMPORT LIBRARY
1. pandas digunakan untuk bekerja dengan data berbentuk tabel (seperti file CSV).
2. TF-IDF (Term Frequency-Inverse Document Frequency) digunakan untuk merepresentasikan data teks secara numerik berdasarkan pentingnya kata dalam dokumen relatif terhadap seluruh dataset.
3. cosine-similarity digunakan untuk membandingkan teks dan mengukur seberapa mirip dua dokumen satu sama lain.
4. tqdm digunakan untuk menampilkan progress bar.

PATH FILE CSV
1. BASE_DIR digunakan untuk menyimpan path dasar di mana file .csv berada.
2. PAPER_DIR digunakan sebagai path ke subfolder bernama "Paper Database" yang ada di dalam BASE_DIR.

LOAD CSV
1. train_df: memuat file CSV train.csv ke dalam DataFrame pandas yang disebut train_df. DataFrame ini akan digunakan untuk menyimpan dan mengelola data pelatihan (training data).
2. metadata_df: memuat file CSV papers_metadata.csv ke dalam DataFrame pandas yang disebut metadata_df.

In [2]:
# Step 1: Buat dictionary paper_id -> text
def load_paper_text(paper_dir):
    paper_texts = {}
    for filename in tqdm(os.listdir(paper_dir), desc="Loading paper texts"):
        if filename.endswith(".txt"):
            paper_id = filename.replace(".txt", "")
            with open(os.path.join(paper_dir, filename), "r", encoding="utf-8") as f:
                text = f.read()
                paper_texts[paper_id] = text
    return paper_texts

paper_texts = load_paper_text(PAPER_DIR)

Loading paper texts: 100%|██████████| 4354/4354 [00:04<00:00, 939.60it/s] 


DICTIONARY
--> load_paper_text digunakan untuk memuat teks dari semua file .txt pada folder Paper Database, mengubahnya menjadi dictionary dengan paper_id sebagai kunci, dan paper_texts sebagai nilai.

In [3]:
# Step 2: Ambil semua paper_id unik yang ada di train.csv
all_ids = set(train_df['paper']).union(set(train_df['referenced_paper']))
filtered_texts = {pid: paper_texts[pid] for pid in all_ids if pid in paper_texts}

PAPER_ID
--> menggabungkan dua kolom dalam train_df yang berisi paper_id: yaitu paper dan referenced_paper dengan menggunakan union() pada set, sehingga mendapatkan semua ID paper yang unik, baik itu sebagai paper utama atau sebagai referensi. 

In [4]:
# Step 3: TF-IDF vectorizer untuk semua teks unik
tfidf = TfidfVectorizer(max_features=5000, stop_words='english')
corpus = list(filtered_texts.values())
paper_ids = list(filtered_texts.keys())
tfidf_matrix = tfidf.fit_transform(corpus)

# Simpan hasil vektor ke dictionary: paper_id -> vector index
paper_to_index = {pid: idx for idx, pid in enumerate(paper_ids)}

TF-IDF Vectorizer
--> digunakan untuk mengubah teks dari setiap paper menjadi representasi numerik, yang memungkinkan model pembelajaran mesin untuk menganalisis dan memproses data teks. Memilih maksimal 5000 kata dari dataset dan menghapus kata-kata yang tidak relevan dengan analisis (stop_words).

TF-IDF_MATRIX
--> merupakan representasi numerik dari teks-teks yang ada.

PAPER_TO_INDEX
--> menyimpan hasil vektor tf-idf ke dalam dictionary.

In [5]:
# Step 4: Contoh membuat fitur cosine similarity untuk train.csv
def compute_cosine(row):
    p1 = row['paper']
    p2 = row['referenced_paper']
    if p1 in paper_to_index and p2 in paper_to_index:
        vec1 = tfidf_matrix[paper_to_index[p1]]
        vec2 = tfidf_matrix[paper_to_index[p2]]
        sim = cosine_similarity(vec1, vec2)[0][0]
        return sim
    else:
        return 0.0  # fallback kalau teks tidak ada

tqdm.pandas(desc="Computing similarity")
train_df['text_similarity'] = train_df.progress_apply(compute_cosine, axis=1)

print(train_df[['paper', 'referenced_paper', 'is_referenced', 'text_similarity']].head())

Computing similarity: 100%|██████████| 410691/410691 [04:30<00:00, 1518.10it/s]

   paper referenced_paper  is_referenced  text_similarity
0  p2128            p3728              0         0.041163
1  p0389            p3811              0         0.057412
2  p1298            p3760              0         0.076167
3  p0211            p1808              0         0.069304
4  p0843            p2964              0         0.039497


COSINE_SIMILARITY
--> nilai antara 0 dan 1 yang menunjukkan kesamaan antara dua teks. Nilai yang lebih tinggi menunjukkan lebih banyak kesamaan, sementara nilai yang lebih rendah menunjukkan lebih sedikit kesamaan.

COMPUTE_COSINE
--> digunakan untuk menghitung cosine similarity antara dua dokumen: paper utama dan paper yang direferensikan.

train_df['text_similarity'] 
--> menambah kolom baru yang berisi hasil cosine similarity untuk setiap baris dalam dataset.

In [6]:
import ast
from tqdm import tqdm

def safe_parse_list(raw):
    """Coba parsing list, fallback ke split semi-colon."""
    if pd.isnull(raw):
        return []
    try:
        # Kalau formatnya ['a', 'b']
        return ast.literal_eval(raw)
    except (ValueError, SyntaxError):
        # Kalau formatnya "a; b; c"
        return [x.strip() for x in raw.split(';') if x.strip()]
    
# Ubah metadata menjadi dictionary: paper_id -> row (as dictionary)
metadata_dict = metadata_df.set_index("paper_id").to_dict(orient="index")

def extract_metadata_features(row):
    p1 = row['paper']
    p2 = row['referenced_paper']
    
    meta1 = metadata_dict.get(p1, {})
    meta2 = metadata_dict.get(p2, {})

    # 1. Selisih tahun publikasi
    year1 = meta1.get('publication_year')
    year2 = meta2.get('publication_year')
    year_diff = year1 - year2 if pd.notnull(year1) and pd.notnull(year2) else 0

    # 2. Cek validitas kutipan
    invalid_citation = int(year_diff < 0) if pd.notnull(year_diff) else 0

    # 3. Overlap penulis
    authors1 = safe_parse_list(meta1.get('authors', ''))
    authors2 = safe_parse_list(meta2.get('authors', ''))
    author_overlap = len(set(authors1) & set(authors2))

    # 4. Overlap konsep
    concepts1 = safe_parse_list(meta1.get('concepts', ''))
    concepts2 = safe_parse_list(meta2.get('concepts', ''))
    concept_overlap = len(set(concepts1) & set(concepts2))

    # 5. Jenis dokumen sama
    type1 = meta1.get('type')
    type2 = meta2.get('type')
    type_match = int(type1 == type2) if type1 and type2 else 0

    return pd.Series({
        'year_diff': year_diff,
        'invalid_citation': invalid_citation,
        'author_overlap': author_overlap,
        'concept_overlap_count': concept_overlap,
        'type_match': type_match
    })

SAFE_PARSE_LIST
--> digunakan untuk mengonversi string yang mungkin berupa daftar (list) atau daftar nilai yang dipisahkan oleh titik koma (;) ke dalam format list Python yang bisa diproses lebih lanjut.

METADATA_DICT
--> mengubah metadata_df menjadi dictionary yang menggunakan paper_id sebagai kunci dan seluruh baris metadata sebagai nilai.

EXTRACT_METADATA_FEATURES
--> digunakan untuk mengekstrak fitur metadata untuk setiap pasangan paper dan paper yang direferensikan dalam dataset dan mengembalikan serangkaian fitur dalam bentuk pd.Series yang berisi nilai untuk setiap fitur yang dihitung: year_diff, invalid_citation, author_overlap, concept_overlap_count, dan type_match.

In [7]:
# Terapkan ke train.csv
tqdm.pandas(desc="Extracting metadata features")
metadata_features = train_df.progress_apply(extract_metadata_features, axis=1)

# Gabungkan dengan train_df
train_df = pd.concat([train_df, metadata_features], axis=1)

Extracting metadata features: 100%|██████████| 410691/410691 [01:31<00:00, 4507.61it/s]


METADATA_FEATURES
--> digunakan untuk menyimpan fungsi extract_metadata_features yang sebelumnya didefinisikan untuk mengekstrak fitur metadata (seperti selisih tahun, validitas kutipan, overlap penulis, dll.).

TRAIN_DF
--> digunakan untuk menyimpan DataFrame baru yang menggabungkan kolom-kolom asli dari train_df dengan kolom-kolom fitur metadata yang baru, sehingga train_df sekarang memiliki kolom tambahan yang berisi fitur metadata.

In [8]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Fitur dan label
features = [
    'text_similarity',
    'year_diff',
    'invalid_citation',
    'author_overlap',
    'concept_overlap_count',
    'type_match'
]

X = train_df[features]
y = train_df['is_referenced']

# Bagi data: train + validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Buat model
model = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False,
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1,
    random_state=42
)

FEATURES
--> daftar kolom yang akan digunakan sebagai fitur (X) untuk model dan fitur-fitur ini sudah diekstrak sebelumnya.

X = train_df[features] 
--> X adalah DataFrame yang berisi fitur yang akan digunakan untuk melatih model. Kolom-kolom yang ada dalam features dipilih dari train_df.

y = train_df['is_referenced']
--> y adalah label yang akan diprediksi oleh model, yaitu apakah suatu paper direferensikan atau tidak. Kolom ini berisi nilai biner (0 atau 1), yang menunjukkan status referensi.

X_train, X_val, y_train, y_val
--> digunakan untuk melatih model dengan menyisihkan 20% dari data untuk validation set, sedangkan 80% digunakan untuk training set. Pembagian data dilakukan secara stratified, artinya pembagian dilakukan dengan cara mempertahankan proporsi label (0 dan 1) yang sama pada training dan validation set.

In [ ]:
# Define parameter grid for hyperparameter tuning
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold

# Define cross-validation strategy
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=xgb.XGBClassifier(
        objective='binary:logistic',
        eval_metric='logloss',
        use_label_encoder=False,
        random_state=42
    ),
    param_distributions=param_grid,
    n_iter=10,  # Number of parameter combinations to try
    scoring='balanced_accuracy',
    cv=cv, 
    verbose=1,
    n_jobs=-1  # Use all available CPU cores
)

# Perform hyperparameter tuning
random_search.fit(X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


C:\Users\ASUS\AppData\Roaming\Python\Python311\site-packages\xgboost\training.py:183: UserWarning: [18:10:32] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


RandomizedSearchCV(cv=StratifiedKFold(n_splits=3, random_state=42, shuffle=True),
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric='logloss',
                                           feature_types=None,
                                           feature_weights=None, gamma=None...
                                           max_leaves=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           multi_strategy=None,
                                           n_estimators=None, n_jobs=None,
                                           num_parallel_tree=None, ...),
                   n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.8, 1.0],
                                        'learning_rate': [0.01, 0.1, 0.2],
                                        'max_depth': [3, 5, 7],
                                        'n_estimators': [50, 100, 150],
                                        'subsample': [0.8, 1.0]},
                   scoring='balanced_accuracy', verbose=1)

PARAM_GRID
--> dictionary yang berisi grid parameter yang ingin diuji dalam proses pencarian hyperparameter. Setiap parameter dalam XGBoost dapat memiliki beberapa nilai yang akan dicoba.

StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
--> metode cross-validation yang memastikan bahwa distribusi label di setiap fold adalah proporsional dengan distribusi label di dataset asli, sehingga menjaga keseimbangan kelas.

RandomizedSearchCV
--> teknik pencarian hyperparameter yang lebih efisien daripada GridSearchCV. Alih-alih mencoba semua kombinasi parameter secara eksak, RandomizedSearchCV mencoba sejumlah kombinasi parameter secara acak (dari grid yang diberikan).

param_distributions=param_grid
--> menyediakan grid parameter yang akan diuji selama pencarian hyperparameter.

random_search.fit(X_train, y_train)
--> melakukan pencarian hyperparameter menggunakan cross-validation pada training set (X_train dan y_train).

In [ ]:
# Get the best model
best_model = random_search.best_estimator_

# Predict on validation set
y_pred = best_model.predict(X_val)

BEST_MODEL
--> proses hyperparameter tuning menggunakan RandomizedSearchCV, kita dapat mengakses model terbaik yang ditemukan selama pencarian (random_search.best_estimator_).

Y_PRED
--> digunakan untuk melakukan prediksi pada data validation set (X_val), menggunakan model terbaik yang telah ditemukan (best_model.predict(X_val).

In [12]:
from sklearn.metrics import matthews_corrcoef
# Evaluate the model
print("Classification Report:\n", classification_report(y_val, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_val, y_pred))

mcc = matthews_corrcoef(y_val, y_pred)
print("Matthews Correlation Coefficient (MCC):", round(mcc, 4))


Classification Report:
               precision    recall  f1-score   support

           0       0.99      1.00      1.00     81281
           1       0.63      0.28      0.39       858

    accuracy                           0.99     82139
   macro avg       0.81      0.64      0.69     82139
weighted avg       0.99      0.99      0.99     82139

Confusion Matrix:
 [[81138   143]
 [  616   242]]
Matthews Correlation Coefficient (MCC): 0.4172


classification_report(y_val, y_pred)
--> fungsi dari scikit-learn yang memberikan rangkuman metrik evaluasi untuk masalah klasifikasi (precision, recall, f1-score, support).

confusion_matrix(y_val, y_pred)
--> fungsi dari scikit-learn yang menghitung matriks kebingungannya dan menunjukkan distribusi true positives (TP), false positives (FP), false negatives (FN), dan true negatives (TN).

matthews_corrcoef(y_val, y_pred)
--> memperhitungkan semua empat elemen dalam matriks kebingungannya (TP, TN, FP, FN) dan memberikan nilai yang antara -1 (prediksi sepenuhnya salah) hingga +1 (prediksi sepenuhnya benar). Nilai 0 menunjukkan prediksi yang tidak lebih baik dari tebak-tebakan acak.


In [ ]:
# Load test data
test_df = pd.read_csv(os.path.join(BASE_DIR, "test.csv"))

# Ekstrak fitur untuk test data
# Karena test data tidak memiliki label, kita hanya perlu fitur-fitur saja
test_features = test_df[['paper', 'referenced_paper']].copy()

# Terapkan fitur yang sama seperti di train data
tqdm.pandas(desc="Extracting metadata features for test data")
test_metadata_features = test_features.progress_apply(extract_metadata_features, axis=1)

# Gabungkan fitur metadata dengan test_df
test_features = pd.concat([test_features, test_metadata_features], axis=1)

# Jangan lupa tambahkan 'text_similarity' (gunakan model TF-IDF yang sudah dibuat)
test_papers = test_features[['paper', 'referenced_paper']]

def compute_test_cosine(row):
    p1 = row['paper']
    p2 = row['referenced_paper']
    if p1 in paper_to_index and p2 in paper_to_index:
        vec1 = tfidf_matrix[paper_to_index[p1]]
        vec2 = tfidf_matrix[paper_to_index[p2]]
        return cosine_similarity(vec1, vec2)[0][0]
    else:
        return 0.0  # fallback

# Hitung similarity untuk test data
test_features['text_similarity'] = test_features.progress_apply(compute_test_cosine, axis=1)


Extracting metadata features for test data: 100%|██████████| 336021/336021 [07:49<00:00, 715.83it/s]  


TEST_DF
--> memuat file test.csv ke dalam DataFrame pandas yang bernama test_df. Data uji ini akan digunakan untuk menguji model setelah pelatihan selesai.

TEST_FEATURES
--> menyimpan dua kolom paper dan referenced_paper dari test_df yang akan digunakan sebagai fitur untuk menghitung similarity antara paper yang mengutip dan yang dikutip.

test_features.progress_apply(extract_metadata_features, axis=1)
--> menerapkan fungsi extract_metadata_features untuk setiap baris pada test_features untuk menghitung fitur metadata untuk setiap pasangan paper dan referenced_paper (selisih tahun, overlap penulis, overlap konsep, dll.). Hasilnya disimpan dalam test_metadata_features.

pd.concat([test_features, test_metadata_features], axis=1)
--> menggabungkan fitur metadata yang diekstrak (test_metadata_features) dengan data test_features yang sudah ada.

TEST_PAPERS
--> membuat DataFrame baru dengan hanya kolom paper dan referenced_paper yang digunakan untuk menghitung similarity antara kedua paper.

COMPUTE_TEST_COSINE
--> digunakan untuk menghitung cosine similarity antara paper dan referenced_paper pada data uji. cosine_similarity(vec1, vec2)[0][0]: menghitung cosine similarity antara kedua vektor tersebut dan mengembalikan nilai skalar kesamaan antara kedua dokumen.

test_features['text_similarity']
--> menambahkan kolom baru text_similarity pada test_features, yang berisi nilai cosine similarity untuk setiap pasangan paper dan referenced_paper pada data uji.

progress_apply(compute_test_cosine, axis=1)
--> menerapkan fungsi compute_test_cosine untuk setiap baris data pada test_features dan menghitung cosine similarity antara setiap paper dan referenced_paper.

In [15]:
# Ekstrak fitur yang digunakan oleh model
X_test = test_features[features]

# Prediksi dengan model yang sudah dilatih
test_pred = best_model.predict(X_test)

# Simpan hasil prediksi dalam dataframe
test_features['is_referenced'] = test_pred

# Tampilkan beberapa hasil prediksi
print(test_features[['paper', 'referenced_paper', 'is_referenced']].head())

   paper referenced_paper  is_referenced
0  p0913            p3488              0
1  p2971            p4337              0
2  p2237            p1610              0
3  p2876            p3212              0
4  p2939            p1901              0


X_TEST
--> DataFrame yang berisi fitur-fitur yang digunakan oleh model untuk membuat prediksi pada data uji. Kolom yang digunakan adalah yang ada dalam features, yang sebelumnya didefinisikan sebagai text_similarity, year_diff, invalid_citation, author_overlap, concept_overlap_count, dan type_match.

best_model.predict(X_test)
--> menggunakan model terbaik yang sudah dipilih (melalui RandomizedSearchCV sebelumnya) untuk memprediksi label pada data uji.

test_features['is_referenced']
--> menambahkan hasil prediksi test_pred ke dalam kolom baru is_referenced di test_features. Kolom ini berisi label prediksi yang menunjukkan apakah paper tersebut direferensikan atau tidak oleh paper lain.

test_features[['paper', 'referenced_paper', 'is_referenced']]
--> memilih kolom-kolom yang ingin ditampilkan, yaitu paper (ID paper utama), referenced_paper (ID paper yang direferensikan), dan is_referenced (Kolom baru yang berisi hasil prediksi (apakah paper utama merujuk paper yang direferensikan)).

In [18]:
# Simpan hasil ke CSV untuk submission
test_features[['is_referenced']].to_csv("C:/Users/ASUS/Downloads/gammafest25/submission9.csv", index=False)

test_features[['is_referenced']]
--> ini memilih kolom is_referenced dari test_features yang berisi hasil prediksi (apakah paper tersebut direferensikan atau tidak).

.to_csv("C:/Users/ASUS/Downloads/gammafest25/submission9.csv", index=False)
--> digunakan untuk menyimpan DataFrame ke dalam file CSV.

In [ ]:
from joblib import dump

# Save the trained model to a file
dump(best_model, "C:/Users/ASUS/Downloads/gammafest25/xgboost_model.joblib")
print("Model saved successfully!")

JOBLIB
--> library yang digunakan untuk serialisasi objek Python, termasuk model machine learning. Library ini dapat digunakan untuk menyimpan model yang telah dilatih ke dalam file.

dump(best_model, ...)
--> digunakan untuk menyimpan best_model (model XGBoost terbaik yang telah dipilih dan dilatih) ke dalam file.